In [0]:
# Load it into a DataFrame
df = spark.read.csv("/databricks-datasets/samples/population-vs-price/data_geo.csv", header=True, inferSchema=True)
df.show(5)

+---------+-------------+-------+----------+------------------------+-----------------------+
|2014 rank|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+---------+-------------+-------+----------+------------------------+-----------------------+
|      101|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|      125|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|      122|       Mobile|Alabama|        AL|                  194675|                  122.5|
|      114|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|       64|Anchorage[19]| Alaska|        AK|                  301010|                   null|
+---------+-------------+-------+----------+------------------------+-----------------------+
only showing top 5 rows



In [0]:
# Load the dataset from DBFS (Databricks Filesystem)
file_path = "/databricks-datasets/samples/population-vs-price/data_geo.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- 2014 rank: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- 2014 Population estimate: integer (nullable = true)
 |-- 2015 median sales price: double (nullable = true)

+---------+-------------+-------+----------+------------------------+-----------------------+
|2014 rank|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+---------+-------------+-------+----------+------------------------+-----------------------+
|      101|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|      125|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|      122|       Mobile|Alabama|        AL|                  194675|                  122.5|
|      114|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|       64|Anchorage[19]| Alaska|        AK|                  30101

In [0]:
# Select relevant columns for analysis
df_selected = df.select(
    "City", "State", "State Code", 
    "2014 Population estimate", "2015 median sales price"
)
df_selected.show(5)

+-------------+-------+----------+------------------------+-----------------------+
|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+-------------+-------+----------+------------------------+-----------------------+
|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|       Mobile|Alabama|        AL|                  194675|                  122.5|
|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|Anchorage[19]| Alaska|        AK|                  301010|                   null|
+-------------+-------+----------+------------------------+-----------------------+
only showing top 5 rows



In [0]:
#dispaly top 10 most popular cities
display(
    df_selected.orderBy('2014 Population estimate', ascending=False)
               .select('City', '2014 Population estimate')
               .limit(10)
)


City,2014 Population estimate
New York[6],8491079
Los Angeles,3928864
Chicago,2722389
Houston[7],2239558
Philadelphia[8],1560297
Phoenix,1537058
San Antonio,1436697
San Diego,1381069
Dallas,1281047
San Jose,1015785


In [0]:
#avg
from pyspark.sql.functions import avg
df_selected.groupBy("State").agg(
    avg("2015 median sales price").alias("Avg Median Price")
).orderBy("Avg Median Price", ascending=False).show()

+--------------------+------------------+
|               State|  Avg Median Price|
+--------------------+------------------+
|             Hawai'i|             699.3|
|          California| 547.9714285714286|
|District of Columbia|             367.8|
|          New Jersey|             350.8|
|            Colorado|333.46666666666664|
|          Washington|266.29999999999995|
|       Massachusetts|             261.8|
|         Connecticut|250.13333333333333|
|                Utah|             243.3|
|       New Hampshire|             237.4|
|              Nevada|             237.3|
|        Rhode Island|             233.3|
|              Oregon|231.53333333333333|
|            Maryland|             223.1|
|           Wisconsin|             210.8|
|           Minnesota|             209.4|
|            Virginia|            201.25|
|             Arizona|             192.1|
|            New York|           185.775|
|        Pennsylvania|             183.8|
+--------------------+------------

In [0]:
#filer cities with population>1 million
df_selected.filter (df_selected["2014 Population estimate"]>1000000).show()

+---------------+------------+----------+------------------------+-----------------------+
|           City|       State|State Code|2014 Population estimate|2015 median sales price|
+---------------+------------+----------+------------------------+-----------------------+
|        Phoenix|     Arizona|        AZ|                 1537058|                  206.1|
|    Los Angeles|  California|        CA|                 3928864|                  434.7|
|      San Diego|  California|        CA|                 1381069|                  510.3|
|       San Jose|  California|        CA|                 1015785|                  900.0|
|        Chicago|    Illinois|        IL|                 2722389|                  192.5|
|    New York[6]|    New York|        NY|                 8491079|                  388.6|
|Philadelphia[8]|Pennsylvania|        PA|                 1560297|                  204.9|
|         Dallas|       Texas|        TX|                 1281047|                  192.5|

In [0]:
# cities with zero or missing values
df_selected.filter(
    (df_selected["2015 median sales price"].isNull()) | 
    (df_selected["2015 median sales price"] == 0)
).select("City").show()

+-------------+
|         City|
+-------------+
|Anchorage[19]|
|     Chandler|
|  Gilbert[20]|
|     Glendale|
|         Mesa|
|       Peoria|
|   Scottsdale|
|     Surprise|
|        Tempe|
|      Antioch|
|  Bakersfield|
|     Berkeley|
|      Burbank|
|     Carlsbad|
|  Chula Vista|
|       Clovis|
|      Concord|
|       Corona|
|   Costa Mesa|
|    Daly City|
+-------------+
only showing top 20 rows

